In [1]:
DATA_PATH = 'C:/Users/korob/Desktop/Lab-2/'

# names of valuable files/folders
train_meta_fname = 'train.csv'
test_meta_fname = 'sample_submission.csv'
train_data_folder = 'train'
test_data_folder = 'test'

In [2]:
pip install git+https://github.com/pytorch/audio

  Cloning https://github.com/pytorch/audio to c:\users\korob\appdata\local\temp\pip-req-build-ro6ogxcg
  Created wheel for torchaudio: filename=torchaudio-0.6.0a0+9835db7-cp37-none-any.whl size=69628 sha256=5d55657d6ec37753f6f1e815ece697bfb6514543462bd84a076683ef3f1b4e1c
  Stored in directory: C:\Users\korob\AppData\Local\Temp\pip-ephem-wheel-cache-3yz2ggyj\wheels\6b\7e\7a\ee4ed533517964e33dc9c0ec29435f8492c976ea85c795527b
Successfully built torchaudio
Note: you may need to restart the kernel to use updated packages.


  Running command git clone -q https://github.com/pytorch/audio 'C:\Users\korob\AppData\Local\Temp\pip-req-build-ro6ogxcg'


In [3]:
import numpy as np
import os
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torchaudio
import torchvision
from torchaudio import transforms
from efficientnet_pytorch import EfficientNet
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from tqdm import tqdm

[('__call__', <function LevelMapper.__call__ at 0x0000011D5BC991E0>), ('__init__', <function LevelMapper.__init__ at 0x0000011D5BC99158>)]
[('__call__', <function BalancedPositiveNegativeSampler.__call__ at 0x0000011D5CD12BF8>), ('__init__', <function BalancedPositiveNegativeSampler.__init__ at 0x0000011D5CD12B70>)]
[('__init__', <function BoxCoder.__init__ at 0x0000011D5CD340D0>), ('decode', <function BoxCoder.decode at 0x0000011D5CD34268>), ('decode_single', <function BoxCoder.decode_single at 0x0000011D5CD342F0>), ('encode', <function BoxCoder.encode at 0x0000011D5CD34158>), ('encode_single', <function BoxCoder.encode_single at 0x0000011D5CD341E0>)]
[('__call__', <function Matcher.__call__ at 0x0000011D5CD26E18>), ('__init__', <function Matcher.__init__ at 0x0000011D5CD26F28>), ('set_low_quality_matches_', <function Matcher.set_low_quality_matches_ at 0x0000011D5CD26EA0>)]
[('__init__', <function ImageList.__init__ at 0x0000011D5CD34598>), ('to', <function ImageList.to at 0x0000011D

In [4]:
import random
import numpy as np

random.seed(42)
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)
torch.backends.cudnn.deterministic = True

In [5]:
df_train = pd.read_csv(os.path.join(DATA_PATH, train_meta_fname))
df_test = pd.read_csv(os.path.join(DATA_PATH, test_meta_fname))
df_train.head(2)

,fname,label
0,8bcbcc394ba64fe85ed4.wav,Finger_snapping
1,00d77b917e241afa06f1.wav,Squeak


In [6]:
n_classes = df_train.label.nunique()
print(n_classes)
classes_dict = {cl:i for i,cl in enumerate(df_train.label.unique())}
df_train['label_encoded'] = df_train.label.map(classes_dict)
df_train.head()

41


,fname,label,label_encoded
0,8bcbcc394ba64fe85ed4.wav,Finger_snapping,0
1,00d77b917e241afa06f1.wav,Squeak,1
2,17bb93b73b8e79234cb3.wav,Electric_piano,2
3,7d5c7a40a936136da55e.wav,Harmonica,3
4,17e0ee7565a33d6c2326.wav,Snare_drum,4


In [7]:
class BaseLineModel(nn.Module):
    
    def __init__(self, sample_rate=16000, n_classes=41):
        super().__init__()
        self.ms = torchaudio.transforms.MelSpectrogram(sample_rate)
#         self.bn1 = nn.BatchNorm2d(1)
        
        self.cnn1 = nn.Conv2d(in_channels=1, out_channels=10, kernel_size=3, padding=1)
        self.cnn3 = nn.Conv2d(in_channels=10, out_channels=3, kernel_size=3, padding=1)
        
        self.features = EfficientNet.from_pretrained('efficientnet-b0')
        # use it as features
#         for param in self.features.parameters():
#             param.requires_grad = False
            
        self.lin1 = nn.Linear(1000, 333)
        
        self.lin2 = nn.Linear(333, 111)
                
        self.lin3 = nn.Linear(111, n_classes)
        
    def forward(self, x):
        x = self.ms(x)
#         x = self.bn1(x)
                
        x = F.relu(self.cnn1(x))
        x = F.relu(self.cnn3(x))
        
        x = self.features(x)

        x = x.view(x.shape[0], -1)
        x = F.relu(x)
        
        x = F.relu(self.lin1(x))
        x = F.relu(self.lin2(x))
        x = self.lin3(x)
        return x
    
    def inference(self, x):
        x = self.forward(x)
        x = F.softmax(x)
        return x

In [8]:
def sample_or_pad(waveform, wav_len=32000):
    m, n = waveform.shape
    if n < wav_len:
        padded_wav = torch.zeros(1, wav_len)
        padded_wav[:, :n] = waveform
        return padded_wav
    elif n > wav_len:
        offset = np.random.randint(0, n - wav_len)
        sampled_wav = waveform[:, offset:offset+wav_len]
        return sampled_wav
    else:
        return waveform
        
class EventDetectionDataset(Dataset):
    def __init__(self, data_path, x, y=None):
        self.x = x
        self.y = y
        self.data_path = data_path
    
    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        path2wav = os.path.join(self.data_path, self.x[idx])
        waveform, sample_rate = torchaudio.load(path2wav, normalization=True)
        waveform = sample_or_pad(waveform)
        if self.y is not None:
            return waveform, self.y[idx]
        return waveform

In [9]:
X_train, X_val, y_train, y_val = train_test_split(df_train.fname.values, df_train.label_encoded.values, 
                                                  test_size=0.2, random_state=42)
train_loader = DataLoader(
                        EventDetectionDataset(os.path.join(DATA_PATH, train_data_folder), X_train, y_train),
                        batch_size=41
                )
val_loader = DataLoader(
                        EventDetectionDataset(os.path.join(DATA_PATH, train_data_folder), X_val, y_val),
                        batch_size=41
                )
test_loader = DataLoader(
                        EventDetectionDataset(os.path.join(DATA_PATH, test_data_folder), df_test.fname.values, None),
                        batch_size=41, shuffle=False
                )

In [15]:
def eval_model(model, eval_dataset):
    model.eval()
    forecast, true_labs = [], []
    with torch.no_grad():
        for wavs, labs in tqdm(eval_dataset):
            wavs, labs = wavs, labs.detach().numpy()
            true_labs.append(labs)
            outputs = model.inference(wavs)
            
            outputs = outputs.detach().cpu().numpy().argmax(axis=1)
            forecast.append(outputs)
    forecast = [x for sublist in forecast for x in sublist]
    true_labs = [x for sublist in true_labs for x in sublist]
    return f1_score(forecast, true_labs, average='macro')

In [16]:
criterion = nn.CrossEntropyLoss()
model = BaseLineModel()
#model = model.cuda()
lr = 1e-3

optimizer = torch.optim.Adam(model.parameters(), lr=lr)

Loaded pretrained weights for efficientnet-b0


In [18]:
n_epoch = 45
best_f1 = 0
for epoch in range(n_epoch):
    model.train()
    for wavs, labs in tqdm(train_loader):
        optimizer.zero_grad()
        wavs, labs = wavs, labs
        outputs = model(wavs)
        loss = criterion(outputs, labs)
        loss.backward()
        optimizer.step()
#     if epoch % 10 == 0:
    f1 = eval_model(model, val_loader)
    f1_train = eval_model(model, train_loader)
    print(f'epoch: {epoch}, f1_test: {f1}, f1_train: {f1_train}')
    if f1 > best_f1:
        best_f1 = f1
        torch.save(model.state_dict(), '../baseline_fulldiv.pt')
        
    lr = lr * 0.95
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

  0%|                                                                                          | 0/111 [00:00<?, ?it/s]

epoch: 0, f1_test: 0.42496657714656466, f1_train: 0.4668509285332465


  0%|                                                                                          | 0/111 [00:00<?, ?it/s]

epoch: 1, f1_test: 0.48255933469060436, f1_train: 0.542973011223873


  0%|                                                                                          | 0/111 [00:00<?, ?it/s]

...

epoch: 42, f1_test: 0.6817848032262303, f1_train: 0.9222123710036141


  0%|                                                                                          | 0/111 [00:00<?, ?it/s]

epoch: 43, f1_test: 0.698170526193128, f1_train: 0.9236982471605797


100%|████████████████████████████████████████████████████████████████████████████████| 111/111 [03:41<00:00,  2.00s/it]

epoch: 44, f1_test: 0.6816149078025162, f1_train: 0.9231748564797034


In [21]:
model_name = 'baseline_fulldiv.pt'
model = BaseLineModel()
model.load_state_dict(torch.load(os.path.join('..', model_name)))
model.eval()
forecast = []
with torch.no_grad():
    for wavs in tqdm(test_loader):
        wavs = wavs
        outputs = model.inference(wavs)
        outputs = outputs.detach().cpu().numpy().argmax(axis=1)
        forecast.append(outputs)
forecast = [x for sublist in forecast for x in sublist]
decoder = {classes_dict[cl]:cl for cl in classes_dict}
forecast = pd.Series(forecast).map(decoder)
df_test['label'] = forecast
df_test.to_csv(f'{model_name}.csv', index=None)

  0%|                                                                                           | 0/93 [00:00<?, ?it/s]

Loaded pretrained weights for efficientnet-b0


c:\users\korob\miniconda3\lib\site-packages\ipykernel_launcher.py:41: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
100%|██████████████████████████████████████████████████████████████████████████████████| 93/93 [03:37<00:00,  2.34s/it]
